In [1]:
import nilearn as nil 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import scipy.stats 
import nibabel as nib 
import os 
%autosave 15  

/opt/anaconda3/envs/glm_pri/lib/python3.7/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


Autosaving every 15 seconds


In [2]:
#data paths and subject list 
data_path = '/data/Wangjing/projects/priority/derivatives/LSS/ses-02_task-learning/outputs_moreclean'
roi_path_base = '/data/Wangjing/projects/priority/derivatives/ROIs'
sub_list = ["%02d" % x for x in range(2, 31)]
runs = ["%02d" % x for x in range(1, 3)]
trials = ["%02d" % x for x in range(1, 55)]

In [2]:
#creating csv files for trial by voxel for run 1 and 2 mpfc 
import os
import nibabel as nib
import numpy as np
import pandas as pd

data_path = '/data/Wangjing/projects/priority/derivatives/LSS/ses-02_task-learning/outputs_moreclean'
roi_path_base = '/data/Wangjing/projects/priority/derivatives/ROIs'
sub_list = ["%02d" % x for x in range(2, 31)]  # For all subjects
runs = ["01", "02"]
trials = ["%02d" % x for x in range(1, 55)]

for sub_id in sub_list:
    for run in runs:
        all_voxels = []

        for trial in trials:
            voxel_values = []

            image_path = os.path.join(data_path, "sub-%s/run-%s/t-%s/cope1.nii.gz" % (sub_id, run, trial))
            roi_path = os.path.join(roi_path_base, "sub-%s/Schaefer/2mm/mpfc_all.nii.gz" % sub_id)
            image = nib.load(image_path)
            roi_image = nib.load(roi_path)

            data = image.get_fdata()
            roi_data = roi_image.get_fdata()

            mask = np.where(roi_data == 1)
            roi_voxels = data[mask].flatten()  # Flattening voxel data into 1D

            voxel_values.extend(roi_voxels)

            all_voxels.append(voxel_values)

        df = pd.DataFrame(all_voxels).transpose()  # Transpose to get voxels as rows
        df.columns = ['trial_%s' % t for t in trials]  # Rename columns
        df.index = ['voxel_%d' % (i+1) for i in range(df.shape[0])]  # Adding index names

        # Ensure the directory exists
        output_dir = f'/data/Kathryn/Projects/Priority/derivatives/rsa_dfs/mpfc/sub-{sub_id}'
        os.makedirs(output_dir, exist_ok=True)

        # Save DataFrame to the specified path for each run
        output_path = os.path.join(output_dir, f'LSS_voxXtrial_mpfc_sub-{sub_id}_run-{run}.csv')
        df.to_csv(output_path)

In [4]:
##extracting voxel and TR data from rest 
import os
import nibabel as nib
import numpy as np
import pandas as pd

data_path = '/data/Wangjing/projects/priority/derivatives/classification'
roi_path_base = '/data/Wangjing/projects/priority/derivatives/ROIs'

# Adjust this list to include identifiers for all subjects
sub_list = ["%02d" % x for x in range(2, 31)]  # Subject 02

all_voxels = {}

for sub_id in sub_list:
    for run_id in ["01", "02", "03"]:
        image_path = os.path.join(data_path, f"sub-{sub_id}/ses-02/task-rest/cleaned_img/cleaned_task-rest_run-{run_id}.nii.gz")  # 4D fMRI data
        roi_path = os.path.join(roi_path_base, f"sub-{sub_id}/Schaefer/2mm/mpfc_all.nii.gz")  # 3D ROI
        image = nib.load(image_path)
        roi_image = nib.load(roi_path)

        data = image.get_fdata()
        roi_data = roi_image.get_fdata()

        mask = np.where(roi_data == 1)

        voxel_list = []
        for t in range(data.shape[-1]):  # For each TR
            roi_voxels = data[..., t][mask]  # Extracting voxel values at the current TR
            voxel_list.append(roi_voxels)
        all_voxels[(sub_id, run_id)] = voxel_list

# Output each subject's voxel data as separate CSVs
for (sub_id, run_id), voxels in all_voxels.items():
    df = pd.DataFrame(voxels).transpose()  # Transpose dataframe
    df.columns = ['TR_%d' % (i+1) for i in range(df.shape[1])]  # Adding column names
    df.index = ['voxel_%d' % (i+1) for i in range(df.shape[0])]  # Adding index names

    output_dir = f'/data/Kathryn/Projects/Priority/derivatives/rsa_dfs/mpfc/sub-{sub_id}'
    os.makedirs(output_dir, exist_ok=True)

    output_path = os.path.join(output_dir, f'LSS_voxXTR_mpfc_sub-{sub_id}_run-{run_id}_day2.csv')
    df.to_csv(output_path)

In [5]:
##extracting voxel and TR data from rest 
import os
import nibabel as nib
import numpy as np
import pandas as pd

data_path = '/data/Wangjing/projects/priority/derivatives/classification'
roi_path_base = '/data/Wangjing/projects/priority/derivatives/ROIs'

# Adjust this list to include identifiers for all subjects
sub_list = ["%02d" % x for x in range(2, 31)]  # Subject 02

all_voxels = {}

for sub_id in sub_list:
    for run_id in ["01"]:
        image_path = os.path.join(data_path, f"sub-{sub_id}/ses-01/task-rest/cleaned_img/cleaned_task-rest_run-{run_id}.nii.gz")  # 4D fMRI data
        roi_path = os.path.join(roi_path_base, f"sub-{sub_id}/Schaefer/2mm/mpfc_all.nii.gz")  # 3D ROI
        image = nib.load(image_path)
        roi_image = nib.load(roi_path)

        data = image.get_fdata()
        roi_data = roi_image.get_fdata()

        mask = np.where(roi_data == 1)

        voxel_list = []
        for t in range(data.shape[-1]):  # For each TR
            roi_voxels = data[..., t][mask]  # Extracting voxel values at the current TR
            voxel_list.append(roi_voxels)
        all_voxels[(sub_id, run_id)] = voxel_list

# Output each subject's voxel data as separate CSVs
for (sub_id, run_id), voxels in all_voxels.items():
    df = pd.DataFrame(voxels).transpose()  # Transpose dataframe
    df.columns = ['TR_%d' % (i+1) for i in range(df.shape[1])]  # Adding column names
    df.index = ['voxel_%d' % (i+1) for i in range(df.shape[0])]  # Adding index names

    output_dir = f'/data/Kathryn/Projects/Priority/derivatives/rsa_dfs/mpfc/sub-{sub_id}'
    os.makedirs(output_dir, exist_ok=True)

    output_path = os.path.join(output_dir, f'LSS_voxXTR_mpfc_sub-{sub_id}_run-{run_id}_day1.csv')
    df.to_csv(output_path)